In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import warnings
import argparse
import os.path as osp
import numpy as np
import time

import tensorflow as tf
import numpy as np

from tf_lab.nips.helper import pclouds_centered_and_half_sphere
from general_tools.simpletons import iterate_in_chunks

from tf_lab.external.Chamfer_EMD_losses.tf_nndistance import nn_distance
from tf_lab.external.Chamfer_EMD_losses.tf_approxmatch import approx_match, match_cost

In [49]:
def normalize_(pc_data):
    pc_data = pc_data - np.expand_dims(np.mean(pc_data, axis = 1), 1)
    dist = np.max(np.sqrt(np.sum(pc_data ** 2, axis = 2)),1)
    dist = np.expand_dims(np.expand_dims(dist, 1), 2)
    pc_data = pc_data / (dist*2)
    return pc_data

def identity(x):
    return x

In [50]:
normalizer = identity

In [51]:
gt_file = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/03001627_train.npz'
gt_data = np.load(gt_file)  # Load Ground-Truth Data.
gt_data = normalizer(gt_data[gt_data.keys()[0]])

In [52]:
epoch = 100

In [53]:
sample_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/l_wgan_chair_disc_512_1024_emd_bneck_128/lam_6/'
sample_file = osp.join(sample_dir, 'epoch_%d.npz' % (epoch,) )
sample_data = np.load(sample_file)
l_wgan_samples = normalizer(sample_data[sample_data.keys()[0]])

In [54]:
sample_file = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/mit_wu_isosurface_extracted/chair.npz'
sample_data = np.load(sample_file)
wu_data = normalizer(sample_data[sample_data.keys()[0]])

In [55]:
sample_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/r_gan/chair_mlp_disc_4_fc_gen_raw_gan_2048_pts/'
sample_file = osp.join(sample_dir, 'epoch_%d.npz' % (epoch,) )
sample_data = np.load(sample_file)
r_gan_samples = normalizer(sample_data[sample_data.keys()[0]])

In [56]:
sample_file = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/gmm/emd/bneck_128_full_20_gaussians.npz'
sample_data = np.load(sample_file)
gmm_samples = normalizer(sample_data[sample_data.keys()[0]])

In [57]:
mmd, dist_matched = minimum_mathing_distance(wu_data[:1000], gt_data[:1000], 1000, normalize=True)
print mmd

0.0338064


In [58]:
mmd, dist_matched = minimum_mathing_distance(gmm_samples[:1000], gt_data[:1000], 1000, normalize=True)
print mmd

0.00294349


In [63]:
mmd, dist_matched = minimum_mathing_distance(l_wgan_samples, gt_data, 5000, normalize=True, verbose=True)
print mmd

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400


KeyboardInterrupt: 

In [60]:
s = time.time()
mmd, dist_matched = minimum_mathing_distance(r_gan_samples[:1000], gt_data[:1000], 1000, normalize=True)
print mmd
e = time.time() - s
print e

0.00231861
30.6182301044


In [ ]:
from geo_tool import Point_Cloud
%matplotlib inline
Point_Cloud(gmm_samples[12]).plot()

In [10]:
# OLD NIPS locations
# sample_dir = '/scr/optas/DATA/nips/our_synthetic_samples/chair_mlp_arch_raw_gan_2048_pts/'
# sample_file = '/scr/optas/DATA/nips/our_synthetic_samples/ground_truth/gt_all_chair.npz'